In [95]:
from openbb_terminal.sdk import openbb
import config_params
import pandas as pd
openbb.keys.fred(key= "f5ad05ac8a52180b33abf7d0976f0428 ", persist=True)


fed_df = openbb.fixedincome.fed(
    start_date=config_params.start_date,
    end_date=config_params.end_date
)

# print(openbb.fixedincome.ameribor(
#     start_date=config_params.start_date,
#     end_date=config_params.end_date,
#     parameter='1_month_term_structure'
# ))

# print(openbb.fixedincome.treasury(
#     short_term=['short'],
#     long_term=['long'],
#     start_date="2023-01-01",
#     end_date="2023-10-01"
# ))

# short_df = pd.read_csv(
#     "https://stats.oecd.org/sdmx-json/data/DP_LIVE/.STINT.TOT.PC_PA.M/OECD?contentType=csv&detail=code&separator=comma&csv-lang=en&startPeriod=2003-10-01&endPeriod=2023-10-01",
#     index_col=5
# )
# s = "https://stats.oecd.org/sdmx-json/data/DP_LIVE/.STINT.TOT.PC_PA.M/OECD?contentType=csv&detail=code&separator=comma&csv-lang=en&startPeriod=2003-10&endPeriod=2023-11"
# d = "https://stats.oecd.org/sdmx-json/data/DP_LIVE/.LTINT.TOT.PC_PA.M/OECD?contentType=csv&detail=code&separator=comma&csv-lang=en&startPeriod=2003-10&endPeriod=2023-10"

# xdiff =""
# ydiff =""
# for x,y in zip(s,d):
#     if x!=y:
#         xdiff+=x
#         ydiff+=y

# print(ydiff)
# print(xdiff)

In [42]:
fed_df.to_csv('FED.csv')

In [127]:
short = pd.read_csv('SHORT.csv')
long = pd.read_csv('LONG.csv')

In [128]:
short = short[short['LOCATION']=='USA']
long = long[long['LOCATION']=='USA']

In [129]:
short.head()

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
178,USA,STINT,TOT,PC_PA,M,2003-10,1.10,NaN
179,USA,STINT,TOT,PC_PA,M,2003-11,1.11,NaN
180,USA,STINT,TOT,PC_PA,M,2003-12,1.10,NaN
181,USA,STINT,TOT,PC_PA,M,2004-01,1.06,NaN
182,USA,STINT,TOT,PC_PA,M,2004-02,1.05,NaN


In [130]:
short = short[['TIME','Value']]
short.columns=['Date','3_months_rate']
long = long[['TIME','Value']]
long.columns=['Date','10_year_rate']


In [131]:
long

,Date,10_year_rate
3985,2003-10,4.29
3986,2003-11,4.30
3987,2003-12,4.27
3988,2004-01,4.15
3989,2004-02,4.08
...,...,...
4220,2023-05,3.57
4221,2023-06,3.75
4222,2023-07,3.90
4223,2023-08,4.17


In [82]:
len(short)

239

In [97]:
fed_df.reset_index(names='Date',inplace=True)

In [111]:
from datetime import datetime

timestamp = fed_df['Date'].values.tolist()[1] // 10**9
print(timestamp)
date = datetime.fromtimestamp(timestamp).
date

1067644800


datetime.datetime(2003, 10, 31, 19, 0)

In [117]:
fed_df['Date'] = pd.to_datetime(fed_df['Date'])
fed_df['Date'].values.tolist()[1]

1067644800000000000

In [125]:
fed_df['Date'] = pd.to_datetime(fed_df['Date']).dt.strftime('%Y-%m')
fed_df

,Date,FEDFUNDS
0,2003-10,1.01
1,2003-11,1.00
2,2003-12,0.98
3,2004-01,1.00
4,2004-02,1.01
...,...,...
235,2023-05,5.06
236,2023-06,5.08
237,2023-07,5.12
238,2023-08,5.33


In [132]:
short.set_index(keys='Date',inplace=True)
long.set_index(keys='Date',inplace=True)
fed_df.set_index(keys='Date',inplace=True)

In [91]:
long

,10_year_rate
Date,
2003-10,4.29
2003-11,4.30
2003-12,4.27
2004-01,4.15
2004-02,4.08
...,...
2023-05,3.57
2023-06,3.75
2023-07,3.90


In [93]:
len(short),len(long),len(fed_df)

(239, 240, 240)

In [118]:
# fed_df.head()


,10_year_rate
Date,
2023-05,3.57
2023-06,3.75
2023-07,3.90
2023-08,4.17
2023-09,4.38


In [133]:
other_data = pd.concat([long,short,fed_df],axis=1)
other_data

,10_year_rate,3_months_rate,FEDFUNDS
Date,,,
2003-10,4.29,1.10,1.01
2003-11,4.30,1.11,1.00
2003-12,4.27,1.10,0.98
2004-01,4.15,1.06,1.00
2004-02,4.08,1.05,1.01
...,...,...,...
2023-05,3.57,5.15,5.06
2023-06,3.75,5.22,5.08
2023-07,3.90,5.35,5.12


In [135]:
other_data.fillna(method='ffill',inplace=True)

In [137]:
other_data.to_csv('Other.csv')

In [1]:
import pandas as pd
import yfinance as yf
import config_params

sp500 = yf.download(tickers=["^RUT"], start=config_params.start_date, end=config_params.end_date, interval="1mo")

bonds = yf.download(tickers=["AGG"], start=config_params.start_date, end=config_params.end_date, interval="1mo")

bonds.reset_index(inplace=True)
sp500.reset_index(inplace=True)

sp500["ticker"] = "^RUT"
bonds["ticker"] = "AGG"
data = pd.concat([bonds, sp500], axis=0)
data['Date'] = pd.to_datetime(data['Date']).dt.strftime('%Y-%m')
other = pd.read_csv('Other.csv')
# data = pd.concat([data,other],axis=0)
data.sort_values(["Date", "ticker"], inplace=True)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [2]:
from black_litterman_env import BlackLittermanEnv

test_data = data[data.Date >= "2022"]
train_data = data[data.Date < "2022"]

env = BlackLittermanEnv(
    data_df=train_data,
    if_confidence=False
)

In [3]:
state,_ = env.reset()

In [4]:
state

array([[1.02150002e+02, 5.56909973e+02],
       [1.03790001e+02, 6.01500000e+02],
       [1.01300003e+02, 5.56909973e+02],
       [5.54215050e+01, 5.80760010e+02],
       [1.05840000e+06, 3.31013500e+10],
       [4.15000000e+00, 1.06000000e+00]])

In [5]:
s,r,t,t,_ = env.step([[1,1,1]])

In [6]:
s

array([[1.02540001e+02, 5.80760010e+02],
       [1.03519997e+02, 5.97080017e+02],
       [1.02230003e+02, 5.64030029e+02],
       [5.59022217e+01, 5.85559998e+02],
       [7.77900000e+05, 2.79856000e+10],
       [4.08000000e+00, 1.05000000e+00]])

In [7]:
r

7463.421274791588

In [8]:
t

False

In [9]:
t

False

In [10]:
env.weights_memory

[[0.4, 0.6], array([1., 0.])]

In [11]:
env.action_space

Box(-1.0, 1.0, (3,), float64)

In [12]:
env.asset_memory

[1000000, 1007463.4212747916]

In [13]:
env.

['10_year_rate', '3_months_rate']